In [18]:
import pandas as pd
from numpy import loadtxt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
from tqdm import trange
from pymystem3 import Mystem
import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import random

Выгружаем исходный набор данных

In [4]:
data = pd.read_csv('datafraem.csv')
data.head()

,Unnamed: 0,class_mark,text_review
0,0,bad,"Оно\nМои фильмы из помойки!Мам, а у нас еще ос..."
1,1,bad,"Оно\nМои фильмы из помойки!Сразу скажу, что я ..."
2,2,bad,Оно\nМои фильмы из помойки!Если сравнивать ста...
3,3,bad,Оно\nМои фильмы из помойки!— Прошу меня извини...
4,4,bad,"Оно\nМои фильмы из помойки!Для начала, опишу х..."


In [5]:
data.dropna(inplace=True)
change_labels = lambda x: 0 if x=='bad' else 1
data['class_mark'] = data['class_mark'].apply(change_labels)
data.head()

,Unnamed: 0,class_mark,text_review
0,0,0,"Оно\nМои фильмы из помойки!Мам, а у нас еще ос..."
1,1,0,"Оно\nМои фильмы из помойки!Сразу скажу, что я ..."
2,2,0,Оно\nМои фильмы из помойки!Если сравнивать ста...
3,3,0,Оно\nМои фильмы из помойки!— Прошу меня извини...
4,4,0,"Оно\nМои фильмы из помойки!Для начала, опишу х..."


Лемматизируем текстики

In [7]:
def Lemmatize(arr: list):
 
    text_nomalization = ' '.join(arr).lower() 

    m = Mystem()
    lemmas = m.lemmatize(text_nomalization)
    
    return lemmas

In [9]:
remove_non_alphabets = lambda x: re.sub(r'[^а-яА-Я]',' ',str(x))

In [10]:
tokenize = lambda x: word_tokenize(x, language = "russian")

In [11]:
ps = PorterStemmer()
stem = lambda w: [ ps.stem(x) for x in w ]

In [ ]:
data['text_review'] = data['text_review'].apply(remove_non_alphabets)
data['text_review'] = data['text_review'].apply(tokenize)
data['text_review'] = data['text_review'].apply(stem)

c = 0
for i in data.index:
    data['text_review'][i] = Lemmatize(data['text_review'][i])
    data["text_review"][i] = [elem for elem in data["text_review"][i] if elem!=' ']
    c+=1

data.dropna(inplace=True)

data.head()

Производим разделение загруженного набора данных на обучающую, тестовую и валидационую выборки (в соотношении 80:10:10). Проверяем, что сформированные выборки сбалансированы

In [16]:
frac_seed = random.randint(0,10)
train_df = data.sample(frac=0.9, random_state=frac_seed, ignore_index=True)
valid_df = data.sample(frac=0.1, random_state=frac_seed, ignore_index=True)
for i in valid_df["text_review"]:
    while i in train_df['text_review']: 
        tmp = data.sample()
        i["text_review"] = tmp['text_review']
        i["class_mark"] = tmp["class_mark"]
     
train_df.to_csv("train_df.csv")
valid_df.to_csv("valid_df.csv")

Создаём модель нейронной сети для решения задачи классификации.

In [ ]:
max_words = 10000
stopWords = stopwords.words('russian')

cv = CountVectorizer(max_features=max_words , stop_words=stopWords)

sparse_matrix = cv.fit_transform(train_df["text_review"]).toarray()
valid_sparse_matrix = cv.fit_transform(valid_df["text_review"]).toarray()
print(sparse_matrix.shape)
print(valid_sparse_matrix.shape)

x_val, x_test, y_val, y_test = train_test_split(valid_sparse_matrix, np.array(valid_df["class_mark"]))

x_train, x_test, y_train, y_test = train_test_split(sparse_matrix, np.array(train_df["class_mark"]),test_size = 0.12, train_size= 0.88)

Описаываем пайплайн предобработки данных. ВАЖНО: что так как ваш вариант предполагает работу с текстом, то необходимо выполниить векторизацию данных (подробности в туториале).

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 100)
        self.linear2 = nn.Linear(100, 10)
        self.linear3 = nn.Linear(10, 6)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

x_train = Variable(torch.from_numpy(x_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()

Написать train loop (цикл обучения). Провести эксперименты по обучению с различными значениями параметров learning rate (скорость обучения) и batch size (размер мини-пакета). Выбрать по 3 значения для learning rate и batch size (итоговое количество экспериментов будет 9).

In [22]:
def expirement(lr, batch_size):
    model = LogisticRegression()
    criterion = nn.CrossEntropyLoss()
    
    data = torch.utils.data.DataLoader(x_train, batch_size = batch_size)
    optimizer = torch.optim.Adam(params=model.parameters() , lr = lr)
    epochs = 10
    model.train()
    loss_values = []
    acc_values = list()
    
    for epoch in range(epochs):
        for elems in data: 
            
            optimizer.zero_grad()
            
            y_pred = model(x_train)
            
            
            loss = criterion(y_pred, y_train)
            loss_values.append(loss.item())
            pred = torch.max(y_pred, 1)[1].eq(y_train).sum()
            acc = pred * 100.0 / len(x_train)
            acc_values.append(acc)
            loss.backward()
            optimizer.step()
    
    epochs_values = list()
    for i in range(0,epochs): epochs_values.append(i)
    
    loss_values_graph = list()
    
    for i in range(0,len(loss_values), len(loss_values)//epochs):
        loss_values_graph.append(loss_values[i])
        
    acc_values_graph = list()
    
    for i in range(0,len(acc_values), len(acc_values)//epochs):
        acc_values_graph.append(acc_values[i])
    
    plt.plot(epochs_values, loss_values_graph)
    plt.title('Loss Value vs Epochs by lr='+str(lr) +' batch_size='+str(batch_size))
    plt.xlabel('Epochs',)
    plt.ylabel('Loss')
    plt.legend(['Loss'])
    plt.show()
    
    plt.plot(epochs_values, acc_values_graph, color = 'g')
    plt.title('Таблица эпох и качества при lr='+str(lr) + 'batch_size='+str(batch_size))
    plt.xlabel('Эпоха',)
    plt.ylabel('Точность')
    plt.legend(['Точность'])
    plt.show()
    
    return model

Для каждого проведенного эксперимента выводим графики для значения функции потерь (ось x - итерация обучения/номер эпохи; ось y - значение функции потерь) и выбранной метрики качества (ось x - итерация обучения/номер эпохи; ось y - значение метрики качества). Графики необходимо выводить как для обучающей, так и для валидационной выборки.

In [ ]:
my_models = list()
lr_list=[0.0001,0.0002,0.0003]
batch_size=[1000,500,100]

for LR in lr_list:
        for BZ in batch_size:
            my_models.append(expirement(LR,BZ))

Оцениваем качество работы модели на тестовой выборке.

In [ ]:
j = 1
lr_number = 0
bs_number = 0
acc_list = list()
for model in my_models:  
    criterion = nn.CrossEntropyLoss()  
    x_test = torch.Tensor(x_test).float()
    y_test=[int(i) for i in y_test]
    y_test = torch.Tensor(y_test).long()
    model.eval()
    with torch.no_grad():
        for i in range(100):
            y_pred = model(x_test)
            loss = criterion(y_pred, y_test)
            pred = torch.max(y_pred, 1)[1].eq(y_test).sum()
        acc = 100*pred/len(x_test)
        acc_list.append(acc)
        print ( 'Number of model = ', j,'LR: ', lr_list[lr_number],'BS: ',batch_size[bs_number] ,' ----> Accuracy : {}%'.format(100*pred/len(x_test)))
        bs_number += 1
        if j%3 == 0: 
            lr_number += 1
            bs_number = 0
        j +=1
print("Max acc: ", float(max(acc_list))  )   

Делаем выводы по полученным результатам проведенных экспериментов. Какую модель из всех полученных стоит использовать

Сохранияем обученную модель

In [ ]:
i = 0
for model in my_models:
    torch.save(model.state_dict(), os.path.join('./', f'model{i}.pt'))
    i+=1

Выполняем повторную инициализацию модели и загрузку весов(где-то там, в мыслях:)). Демонстрируем работоспособность модели(снова там же:)) (пропустите через нее какой-то отзыв/рецензию и выведите результат)